🪶DATA GATHERING

In [1]:
#import all the necessary packages
import pyodbc
from dotenv import dotenv_values
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import seaborn as sns


In [2]:
import matplotlib.pyplot as plt
from scipy import stats
import statistics as staticmethod
from scipy.stats import f_oneway


In [3]:
environment_variables = dotenv_values('.env')
database = environment_variables.get("DATABASE")
server = environment_variables.get("SERVER")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")

connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection"

In [4]:
connection = pyodbc.connect(connection_string)

In [5]:
#querying data for 2020 from DB

query = "Select * from LP1_startup_funding2020"

data = pd.read_sql(query, connection)

In [6]:
data.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,column10
0,Aqgromalin,2019.0,Chennai,AgriTech,Cultivating Ideas for Profit,"Prasanna Manogaran, Bharani C L",Angel investors,200000.0,None,None
1,Krayonnz,2019.0,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,"Saurabh Dixit, Gurudutt Upadhyay",GSF Accelerator,100000.0,Pre-seed,None
2,PadCare Labs,2018.0,Pune,Hygiene management,Converting bio-hazardous waste to harmless waste,Ajinkya Dhariya,Venture Center,NaN,Pre-seed,None
3,NCOME,2020.0,New Delhi,Escrow,Escrow-as-a-service platform,Ritesh Tiwari,"Venture Catalysts, PointOne Capital",400000.0,None,None
4,Gramophone,2016.0,Indore,AgriTech,Gramophone is an AgTech platform enabling acce...,"Ashish Rajan Singh, Harshit Gupta, Nishant Mah...","Siana Capital Management, Info Edge",340000.0,None,None


Checking for missing data

In [7]:
data.isnull().sum()

Company_Brand       0
Founded           213
HeadQuarter        94
Sector             13
What_it_does        0
Founders           12
Investor           38
Amount            254
Stage             464
column10         1053
dtype: int64

Checking for duplicate data

In [8]:
data.duplicated("Company_Brand").sum()

150

In [9]:
data.shape

(1055, 10)

In [10]:
data_drop_dup = data.drop_duplicates('Company_Brand')

In [11]:
data_drop_dup.shape

(905, 10)

In [12]:
data_drop_dup.isnull().sum()

Company_Brand      0
Founded          168
HeadQuarter       78
Sector             6
What_it_does       0
Founders          12
Investor          37
Amount           230
Stage            398
column10         903
dtype: int64

In [13]:
#drop column10
data_drop_dup = data_drop_dup.drop('column10', axis=1)

In [14]:
data_drop_dup.dtypes

Company_Brand     object
Founded          float64
HeadQuarter       object
Sector            object
What_it_does      object
Founders          object
Investor          object
Amount           float64
Stage             object
dtype: object

In [15]:
data_drop_dup.tail(10)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
1043,Quicko,NaN,Ahmedabad,Taxation,Online tax planning and filing platform,Vishvajit Sonagara,"Zerodha fintech fund, Rainmatter",280000.0,None
1044,Satin Creditcare,NaN,Gurgaon,Fintech,A micro finance company,None,Austrian Bank,15000000.0,None
1045,XpressBees,2015.0,None,Logistics,Provides end to end supply chain solutions,"Supam Maheshwari, Amitava Saha",Alibaba,10000000.0,None
1046,FarmERP,2001.0,Pune,Agritech,Smart agriculture management ERP software plat...,"Santosh Shinde, Sanjay Borkar",TechnoGen,NaN,Series A
1047,Wealth Bucket,2018.0,None,Fintech,An online platform for mutual fund investments,"Himanshu Jain, Pulkit Jain","NorthStar, Vinod Khatumal",3000000.0,None
1049,Fashor,2017.0,Chennai,Fashion,Women’s fashion and apparel,"Vikram Kankaria, Priyanka Kankaria",Sprout venture partners,1000000.0,Pre Series A
1050,Leverage Edu,NaN,Delhi,Edtech,AI enabled marketplace that provides career gu...,Akshay Chaturvedi,"DSG Consumer Partners, Blume Ventures",1500000.0,None
1051,EpiFi,NaN,None,Fintech,It offers customers with a single interface fo...,"Sujith Narayanan, Sumit Gwalani","Sequoia India, Ribbit Capital",13200000.0,Seed Round
1052,Purplle,2012.0,Mumbai,Cosmetics,Online makeup and beauty products retailer,"Manish Taneja, Rahul Dash",Verlinvest,8000000.0,None
1054,Pando,2017.0,Chennai,Logitech,Networked logistics management software,"Jayakrishnan, Abhijeet Manohar",Chiratae Ventures,9000000.0,Series A


In [16]:
data_drop_dup['Amount'].mean()

128451352.15407407

In [17]:
data_drop_dup['Amount'] = data_drop_dup.Amount.fillna(data_drop_dup['Amount'].mean())

In [18]:
data_drop_dup.tail(10)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
1043,Quicko,NaN,Ahmedabad,Taxation,Online tax planning and filing platform,Vishvajit Sonagara,"Zerodha fintech fund, Rainmatter",2.800000e+05,None
1044,Satin Creditcare,NaN,Gurgaon,Fintech,A micro finance company,None,Austrian Bank,1.500000e+07,None
1045,XpressBees,2015.0,None,Logistics,Provides end to end supply chain solutions,"Supam Maheshwari, Amitava Saha",Alibaba,1.000000e+07,None
1046,FarmERP,2001.0,Pune,Agritech,Smart agriculture management ERP software plat...,"Santosh Shinde, Sanjay Borkar",TechnoGen,1.284514e+08,Series A
1047,Wealth Bucket,2018.0,None,Fintech,An online platform for mutual fund investments,"Himanshu Jain, Pulkit Jain","NorthStar, Vinod Khatumal",3.000000e+06,None
1049,Fashor,2017.0,Chennai,Fashion,Women’s fashion and apparel,"Vikram Kankaria, Priyanka Kankaria",Sprout venture partners,1.000000e+06,Pre Series A
1050,Leverage Edu,NaN,Delhi,Edtech,AI enabled marketplace that provides career gu...,Akshay Chaturvedi,"DSG Consumer Partners, Blume Ventures",1.500000e+06,None
1051,EpiFi,NaN,None,Fintech,It offers customers with a single interface fo...,"Sujith Narayanan, Sumit Gwalani","Sequoia India, Ribbit Capital",1.320000e+07,Seed Round
1052,Purplle,2012.0,Mumbai,Cosmetics,Online makeup and beauty products retailer,"Manish Taneja, Rahul Dash",Verlinvest,8.000000e+06,None
1054,Pando,2017.0,Chennai,Logitech,Networked logistics management software,"Jayakrishnan, Abhijeet Manohar",Chiratae Ventures,9.000000e+06,Series A


In [19]:
data_drop_dup.isnull().sum()

Company_Brand      0
Founded          168
HeadQuarter       78
Sector             6
What_it_does       0
Founders          12
Investor          37
Amount             0
Stage            398
dtype: int64

In [58]:
#replaceing the missing stage with mode stage for the year
mode_value_0 = data_drop_dup['Stage'].mode()[0]
mode_value_0

'Series A'

In [59]:
#replace the missing stages with the most repeated stage
data_drop_dup['Stage'].fillna(mode_value_0, inplace=True)

In [60]:
data_drop_dup.isnull().sum()

Company_Brand      0
Founded          168
HeadQuarter       78
Sector             6
What_it_does       0
Founders          12
Investor          37
Amount             0
Stage              0
dtype: int64

2021 Data

In [20]:
#querying data for 2021 from DB

query = "Select * from LP1_startup_funding2021"

data_21 = pd.read_sql(query, connection)

In [21]:
data_21.head(10)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","$1,200,000",Pre-series A
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","$120,000,000",None
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital","$30,000,000",Series D
3,Bizongo,2015.0,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital","$51,000,000",Series C
4,FypMoney,2021.0,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal","$2,000,000",Seed
5,Urban Company,2014.0,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,"Abhiraj Singh Bhal, Raghav Chandra, Varun Khaitan",Vy Capital,"$188,000,000",None
6,Comofi Medtech,2018.0,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,Gururaj KB,"CIIE.CO, KIIT-TBI","$200,000",None
7,Qube Health,2016.0,Mumbai,HealthTech,India's Most Respected Workplace Healthcare Ma...,Gagan Kapur,Inflection Point Ventures,Undisclosed,Pre-series A
8,Vitra.ai,2020.0,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Akash Nidhi PS,Inflexor Ventures,Undisclosed,None
9,Taikee,2010.0,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",None,"$1,000,000",None


In [22]:
data_21.isnull().sum()

Company_Brand      0
Founded            1
HeadQuarter        1
Sector             0
What_it_does       0
Founders           4
Investor          62
Amount             3
Stage            428
dtype: int64

In [23]:
#finding the total duplicate in Company_Brand column
data_21.duplicated("Company_Brand").sum()

176

In [24]:
data_21.shape

(1209, 9)

In [25]:
#drop the duplicates
data_drop_dup_21 = data_21.drop_duplicates('Company_Brand')

In [26]:
data_drop_dup_21.shape

(1033, 9)

In [27]:
data_drop_dup_21.isnull().sum()

Company_Brand      0
Founded            1
HeadQuarter        1
Sector             0
What_it_does       0
Founders           4
Investor          58
Amount             3
Stage            366
dtype: int64

In [28]:
data_drop_dup_21.dtypes

Company_Brand     object
Founded          float64
HeadQuarter       object
Sector            object
What_it_does      object
Founders          object
Investor          object
Amount            object
Stage             object
dtype: object

In [29]:
#removing the "$" sign in the amount column
data_drop_dup_21['Amount'] = data_drop_dup_21['Amount'].str.replace("$","")
data_drop_dup_21.head(10)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","1,200,000",Pre-series A
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","120,000,000",None
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital","30,000,000",Series D
3,Bizongo,2015.0,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital","51,000,000",Series C
4,FypMoney,2021.0,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal","2,000,000",Seed
5,Urban Company,2014.0,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,"Abhiraj Singh Bhal, Raghav Chandra, Varun Khaitan",Vy Capital,"188,000,000",None
6,Comofi Medtech,2018.0,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,Gururaj KB,"CIIE.CO, KIIT-TBI","200,000",None
7,Qube Health,2016.0,Mumbai,HealthTech,India's Most Respected Workplace Healthcare Ma...,Gagan Kapur,Inflection Point Ventures,Undisclosed,Pre-series A
8,Vitra.ai,2020.0,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Akash Nidhi PS,Inflexor Ventures,Undisclosed,None
9,Taikee,2010.0,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",None,"1,000,000",None


In [30]:
#removing the commas in the Amounts column
data_drop_dup_21['Amount'] = data_drop_dup_21['Amount'].str.replace(",","")
data_drop_dup_21.head(10)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First",1200000,Pre-series A
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management",120000000,None
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital",30000000,Series D
3,Bizongo,2015.0,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital",51000000,Series C
4,FypMoney,2021.0,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal",2000000,Seed
5,Urban Company,2014.0,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,"Abhiraj Singh Bhal, Raghav Chandra, Varun Khaitan",Vy Capital,188000000,None
6,Comofi Medtech,2018.0,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,Gururaj KB,"CIIE.CO, KIIT-TBI",200000,None
7,Qube Health,2016.0,Mumbai,HealthTech,India's Most Respected Workplace Healthcare Ma...,Gagan Kapur,Inflection Point Ventures,Undisclosed,Pre-series A
8,Vitra.ai,2020.0,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Akash Nidhi PS,Inflexor Ventures,Undisclosed,None
9,Taikee,2010.0,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",None,1000000,None


In [31]:
#finding the unique values in the Amount column
unique_values_21 = data_drop_dup_21['Amount'].unique()
unique_values_21

array(['1200000', '120000000', '30000000', '51000000', '2000000',
       '188000000', '200000', 'Undisclosed', '1000000', '3000000',
       '100000', '700000', '9000000', '40000000', '49000000', '400000',
       '300000', '25000000', '160000000', '150000', '1800000', '5000000',
       '850000', '53000000', '500000', '1100000', '6000000', '800000',
       '10000000', '21000000', '7500000', '26000000', '7400000',
       '1500000', '600000', '800000000', '17000000', '3500000',
       '15000000', '215000000', '2500000', '350000000', '5500000',
       '83000000', '110000000', '500000000', '65000000', '150000000000',
       '300000000', '2200000', '140000000', '4000000', '13000000', None,
       '9500000', '8000000', 'Upsparks', '12000000', '1700000',
       '150000000', '100000000', '225000000', '6700000', '1300000',
       '20000000', '250000', '52000000', '3800000', '17500000',
       '42000000', '2300000', '7000000', '450000000', '8500000',
       '37000000', '44000000', '770000', '12500

In [32]:
data_drop_dup_21['numbers'] = data_drop_dup_21['Amount'].str.extract('(\d+)', expand=False)
data_drop_dup_21['text'] = data_drop_dup_21['Amount'].str.extract('([^\d]+)', expand=False)

In [33]:
data_drop_dup_21.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,numbers,text
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First",1200000,Pre-series A,1200000,NaN
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management",120000000,None,120000000,NaN
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital",30000000,Series D,30000000,NaN
3,Bizongo,2015.0,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital",51000000,Series C,51000000,NaN
4,FypMoney,2021.0,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal",2000000,Seed,2000000,NaN


In [34]:
data_drop_dup_21['text'].unique()

array([nan, 'Undisclosed', None, 'Upsparks', 'Series C', 'Seed',
       'undisclosed', 'ah! Ventures', 'Pre-series A',
       'ITO Angel Network LetsVenture', 'JITO Angel Network LetsVenture'],
      dtype=object)

In [35]:
#drop Amount Column
data_drop_dup_21 = data_drop_dup_21.drop('Amount', axis=1)

In [36]:
data_drop_dup_21.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Stage,numbers,text
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First",Pre-series A,1200000,NaN
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management",None,120000000,NaN
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital",Series D,30000000,NaN
3,Bizongo,2015.0,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital",Series C,51000000,NaN
4,FypMoney,2021.0,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal",Seed,2000000,NaN


In [37]:
#rename numbers column to Amount
data_drop_dup_21.rename(columns = {'numbers':'Amount'},inplace=True)

In [38]:
print(data_drop_dup_21.columns)

Index(['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does',
       'Founders', 'Investor', 'Stage', 'Amount', 'text'],
      dtype='object')


In [39]:
#convert Amount into integer
data_drop_dup_21['Amount'] = data_drop_dup_21['Amount'].astype(float)

In [40]:
data_drop_dup_21.isnull().sum()

Company_Brand      0
Founded            1
HeadQuarter        1
Sector             0
What_it_does       0
Founders           4
Investor          58
Stage            366
Amount           142
text             895
dtype: int64

In [41]:
data_drop_dup_21.shape

(1033, 10)

In [42]:
data_drop_dup_21['Amount'].mean()

189016415.2637486

In [48]:
#replace missing Amount with the mean amount for the year
data_drop_dup_21['Amount'] = data_drop_dup_21.Amount.fillna(data_drop_dup_21['Amount'].mean())

In [50]:
data_drop_dup_21.head(20)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Stage,Amount,text
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First",Pre-series A,1.200000e+06,NaN
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management",None,1.200000e+08,NaN
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital",Series D,3.000000e+07,NaN
3,Bizongo,2015.0,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital",Series C,5.100000e+07,NaN
4,FypMoney,2021.0,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal",Seed,2.000000e+06,NaN
5,Urban Company,2014.0,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,"Abhiraj Singh Bhal, Raghav Chandra, Varun Khaitan",Vy Capital,None,1.880000e+08,NaN
6,Comofi Medtech,2018.0,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,Gururaj KB,"CIIE.CO, KIIT-TBI",None,2.000000e+05,NaN
7,Qube Health,2016.0,Mumbai,HealthTech,India's Most Respected Workplace Healthcare Ma...,Gagan Kapur,Inflection Point Ventures,Pre-series A,1.890164e+08,Undisclosed
8,Vitra.ai,2020.0,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Akash Nidhi PS,Inflexor Ventures,None,1.890164e+08,Undisclosed
9,Taikee,2010.0,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",None,None,1.000000e+06,NaN


In [51]:

# Find the mode of the Founded column
mode_value = data_drop_dup_21['Founded'].mode()[0]
mode_value

2020.0

In [52]:
#replace the missing year with the most repeated year
data_drop_dup_21['Founded'].fillna(mode_value, inplace=True)

In [53]:
data_drop_dup_21.isnull().sum()

Company_Brand      0
Founded            0
HeadQuarter        1
Sector             0
What_it_does       0
Founders           4
Investor          58
Stage            366
Amount             0
text             895
dtype: int64

In [54]:
# Find the mode of the Stage column
mode_value1 = data_drop_dup_21['Stage'].mode()[0]
mode_value1

'Seed'

In [55]:
#replace the missing stages with the most repeated stage
data_drop_dup_21['Stage'].fillna(mode_value1, inplace=True)

In [56]:
data_drop_dup_21.isnull().sum()

Company_Brand      0
Founded            0
HeadQuarter        1
Sector             0
What_it_does       0
Founders           4
Investor          58
Stage              0
Amount             0
text             895
dtype: int64

In [57]:
#drop Amount Column
data_drop_dup_21 = data_drop_dup_21.drop('text', axis=1)